# 입문자를 위한, 파이썬/R 데이터 분석

## Today's mission

- 워드클라우드 : 다음뉴스 키워드 검색 및 텍스트 분석
- 워드클라우드 : soynlp 라이브러리 활용 명사 추출

## 워드클라우드 : soynlp 라이브러리 활용 명사 추출
* soynlp(https://github.com/lovit/soynlp) 로 토큰화하고 명사를 추출해 워드클라우드 그려보기

In [1]:
# !pip install soynlp
# !pip show soynlp
# !pip install wordcloud
# !pip install koreanize-matplotlib

In [2]:
import pandas as pd
import numpy as np
import re
#import koreanize_matplotlib
import os
import platform
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# 데이터 불러오기
*  데이터 출처 : https://github.com/akngs/petitions   (과거데이터만)

In [3]:
df = pd.read_csv(f"data/petition.csv", index_col="article_id",
                        parse_dates=['start', 'end'])
df.shape

(377756, 7)

In [4]:
df.shape

(377756, 7)

In [5]:
df.tail()

,start,end,answered,votes,category,title,content
article_id,,,,,,,
492039,2019-01-09,2019-02-08,0,48,인권/성평등,박종철 의원. 폭행구속수사 및 의원직 박탈 청원합니다,나라돈으로 해외연수가서 현지 동반 가이드를 때려 안경 파편이 얼굴이 꽂혔답니다. C...
492040,2019-01-09,2019-02-08,0,5,정치개혁,각 정상들은 들으시요 국회의원을 학력과 무관한 양심 및 윤리가 있는 서민이 되어...,각 정당은 현재 국회의원 개개인을 심사하면 지탄받은 사람이 국\n회를 장악하고 있습...
492041,2019-01-09,2019-02-08,0,1,외교/통일/국방,남한땅에 옥류관을 오픈해주세요,말그대로 옥류관을 여기서 열면 진짜 재미있고 신나는 일이 일어날것 같은 느낌이 듭니...
492042,2019-01-09,2019-02-08,0,4,정치개혁,임종석실장님 수고많으셨습니다.,범죄정권이후 많은 어려움을 갖고 시작한 국민의정부.\n저급한 자칭 보수단체와 한국당...
492043,2019-01-09,2019-02-08,0,1,행정,예천군과 환경부를 규탄합니다. 어불성설인 가축사육 관련 법규를 개정해주세요!,해당 사건이 발생한 곳은 요즘은 매체에서 매일 나오는 곳으로 '군의원의 외유성 해외...


In [6]:
df.head()

,start,end,answered,votes,category,title,content
article_id,,,,,,,
21,2017-08-19,2017-11-17,0,9,안전/환경,스텔라 데이지호에 대한 제안입니다.,스텔라 데이지호에 대한 제안입니다.\n3월31일 스텔라 데이지호가 침몰하고 5달째가...
22,2017-08-19,2017-11-17,0,17,기타,비리제보처를 만들어주세요.,현 정부에 국민들이 가장 원하는 것은 부패척결입니다. 우리 사회에 각종 비리들이 ...
23,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."
24,2017-08-19,2017-08-26,0,53,일자리,공공기관 무조건적인 정규직전환을 반대합니다.,현정부에서 정규직 일자리를 늘리는 것에 찬성합니다. 그런데 공공기관 비정규직들은 인...
25,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."


## 자신의 관심사에 맞는 단어로 데이터를 가져온다.

In [7]:
p = r'.*(돌봄|육아|초등|보육).*'
care = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
care.shape

(14960, 7)

In [8]:
care.head(2)

,start,end,answered,votes,category,title,content
article_id,,,,,,,
24,2017-08-19,2017-08-26,0,53,일자리,공공기관 무조건적인 정규직전환을 반대합니다.,현정부에서 정규직 일자리를 늘리는 것에 찬성합니다. 그런데 공공기관 비정규직들은 인...
36,2017-08-19,2017-08-26,0,1,인권/성평등,한국채식인구 100만명. 학교 급식 및 군대에서 현미채식 선택권을 보장해주십시오!,문재인 대통령님과 각 정부 인사분들께 마음속 깊이 존경과 감사를 표합니다. 대한민국...


In [9]:
sample_index = 24

In [10]:
sample_title = care.loc[sample_index, 'title']
sample_title

'공공기관 무조건적인 정규직전환을 반대합니다.'

### sample_content 내용 확인

In [11]:
sample_content = care['content'][sample_index]
sample_content

'현정부에서 정규직 일자리를 늘리는 것에 찬성합니다. 그런데 공공기관 비정규직들은 인맥으로 들어온 경우가 많습니다. 자질이 안되는데도 정규직이 된다면 그 피해는 국민들에게 돌아갈것입니다. 현재 공공기관 정규직들은 100대1의 경쟁률을 뚫고 들어온 경우도 있습니다. 지금도 노량진에서 수많은 청춘들이 공부를 하고 있죠. 기존 비정규직들을 무조건적으로 무기직 전환한다면 또 다른 정유라 탄생이고 역차별입니다. 새로 필요로 하는 신규채용부터 공채절차를 거쳐 무기직 전환해야합니다. 예전에 공무원 기능직의 일반직 전환, 초등학교에 중초교사 임용 등이 그 예죠. 실제 일하는 곳에서는 그분들로 인한 업무처리 미흡으로 문제가 되고 있습니다. 사립학교는 인건비를 국가에서 주니 사립학교도 국가에서 공채해야 합니다. 부디 사례 하나하나를 보아가며 처리해주시고 전체 정규직 숫자 전환만 보며 공약실천을 무리하게 하지 말아주세요.  국민들은 정의로운 나라를 원합니다. 역차별이 아닌 공정한 채용이 되게 해주세요.'

## 토큰화

Soynlp에서 제공하는 RegexTokenizer를 사용하여 토큰화 

# 토큰화

In [13]:
#!pip install soynlp

In [12]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

ModuleNotFoundError: No module named 'soynlp'

In [ ]:
tokened_title = tokenizer.tokenize(sample_title)
tokened_title

In [ ]:
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:20]

In [ ]:
print(len(tokened_title))
print(len(tokened_content))

# 텍스트 데이터 전처리 
* 개행문자(줄바꿈 문자) 제거

In [ ]:
care['content'].head()

In [ ]:
content_text = care['content'].str.replace("\\\\n", " ", regex=True)
content_text = content_text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 a-zA-Z]", " ", regex=True)

In [ ]:
content_text.head()

In [ ]:
tokens = content_text.apply(tokenizer.tokenize)
tokens[:3]

In [ ]:
tokens[sample_index][:10]

In [ ]:
content_text.head()

## 워드 클라우드 그리기

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
stopwords = ['하지만', '그리고', '그런데', '저는','제가',
             '그럼', '이런', '저런', '합니다',
             '많은', '많이', '정말', '너무'] 

- [WordCloud 오류 해결](https://velog.io/@jaeho99/%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%97%94%EC%A7%80%EB%8B%88%EC%96%B4%EB%A7%81-%EB%8D%B0%EB%B8%8C%EC%BD%94%EC%8A%A4-2%EA%B8%B0-TIL3%EC%A3%BC%EC%B0%A8-06-%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-%EC%9B%B9-%EB%8B%A4%EB%A3%A8%EA%B8%B0%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%8B%9C%EA%B0%81%ED%99%94)

In [ ]:
#!pip install --upgrade pip
#!pip install --upgrade Pillow

In [ ]:
def display_word_cloud(data, width=1200, height=500):
    
    word_draw = WordCloud(
        font_path=r"data/malgun.ttf",
        width=width, height=height,
        stopwords=stopwords, 
        background_color="white",
        random_state=42
    )
    word_draw.generate(data)

    plt.figure(figsize=(15, 7))
    plt.imshow(word_draw)
    plt.axis("off")
    plt.show()

In [ ]:
# display_word_cloud 함수 안에 font_path 위치
display_word_cloud(' '.join(content_text))

출력 결과를 보면 '수'나 '및'과 같이 문서의 의미를 특징 짓지 않는 불용어(STOPWORD)가 너무 많다는 것을 알 수 있다.


## 명사만 추출하여 시각화

In [ ]:
from soynlp.noun import LRNounExtractor

- 주피터 노트북에서 실행 시간을 측정하는 매직 명령어   
    - %%time : 전체 셀의 실행 시간을 측정. 따라서 셀에 포함된 모든 코드의 실행 시간을 측정.  
    - %time : 한 줄의 코드나 한 줄의 표현식의 실행 시간을 측정. 따라서 코드 한 줄의 실행 시간만을 측정.

In [ ]:
%%time
noun_extractor = LRNounExtractor(verbose=True)
noun_extractor.train(content_text)
nouns = noun_extractor.extract()

In [ ]:
nouns_text = " ".join(list(nouns.keys()))
nouns_text[:100]

In [ ]:
# 추출된 명사를 워크 클라우드를 통해 시각화한다.
%time
display_word_cloud(nouns_text)